# Lab and Chart Events
## Data Extraction and Preparation

### Setup
We are keeping the MIMIC III tables in AWS Athena to use standard SQL queries to extract the data.
The outputs are saved into *pickle* files for further use.

Here we setup the connection to Athena by indicating the S3 buckets were the MIMIC III files (in *parquet* format).

In [1]:
!pip install PyAthena
from pyathena import connect
from pyathena.pandas.util import as_pandas


# Import libraries
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import boto3
from botocore.client import ClientError
from IPython.display import display, HTML
import pickle
%matplotlib inline


s3 = boto3.resource('s3')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name
athena_query_results_bucket = 'aws-athena-query-results-'+account_id+'-'+region

try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
except ClientError:
    bucket = s3.create_bucket(Bucket=athena_query_results_bucket)
    print('Creating bucket '+athena_query_results_bucket)
cursor = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp').cursor()

## Diabetic patients cohort

We have two cohorts.  The diabetic patients in the MIMIC III database have class imbalanced for 'dead' (1217) and 'alive' (8605) outcomes.

To overcome this issue we created a *balanced* cohort by means of oversampling (see details in `sql_queries.ipynb`).

In [2]:
cohort_table = "default.diabetic_patients_cohort"     ## FULL COHORT
subject_id_col = "subject_id"                         ## FULL COHORT
prefix = ""                                           ## FULL COHORT

#cohort_table = "default.train_cohort2"                ## BALANCED COHORT - TRAINING
#prefix = "balanced_train_"                            ## BALANCED COHORT - TRAINING
#subject_id_col = "new_subject_id"                      ## BALANCED COHORT - TRAINING

#cohort_table = "default.test_cohort"                   ## BALANCED COHORT - TESTING
#prefix = "balanced_test_"                              ## BALANCED COHORT - TESTING
#subject_id_col = "subject_id"                      ## BALANCED COHORT - TESTING

query='select * from default.diabetic_patients_cohort order by subject_id'
cursor.execute(query)
patients = as_pandas(cursor)

query='select mortality_flag, count (distinct ' + subject_id_col + ') from ' + cohort_table +' group by mortality_flag'
cursor.execute(query)
mf = as_pandas(cursor)
mf.head()

,mortality_flag,_col1
0,1,1217
1,0,8605


## Selected ICD9 codes

The 17 APACHE III features and the 3 diabetic specific features (HbA1c, blood glucose and serum creatinine) map to many ICD9 codes.

Also, using the attention weights we selected a subset of the features that got the most attention from the model (named *relevant* here).

In [3]:
chartevents_codes = [
    ['Capillary refill rate', 3348, 224308, 223951, 8377, 115],
    ['Diastolic blood pressure', 8364, 225310, 228151, 8555, 8368, 220051, 8502, 8503, 8504, 8505, 8506, 8507, 8508, 153, 8440, 224643, 227242, 8441, 220180, 8444, 8445, 8446, 8448, 220060],
    ['Fraction inspired oxygen', 7146, 226767, 227035, 228192, 228193, 228232],
    ['Glascow coma scale eye opening', 184, 220739],
    ['Glascow coma scale motor response', 223901, 226757],
    ['Glascow coma scale total', 198],
    ['Glascow coma scale verbal response', 223900, 226758],
    ['Glucose', 3744, 3745, 1310, 807, 1529, 811, 220621, 226537, 3447, 225664],
    ['Heart Rate', 211, 220045],
    ['Height', 226730],
    ['Mean blood pressure', 225312, 52, 6702, 220052, 6927, 3312, 3314, 3316, 7618, 3318, 3320, 3322, 7620, 7622, 3324, 5702, 443, 456, 220181],
    ['Oxygen saturation', 0],
    ['Respiratory rate', 220210, 618, 224688, 224690, 224689, 619],
    ['Systolic blood pressure', 51, 225309, 220050, 3313, 3315, 3317, 3319, 3321, 3323, 3325, 442, 224167, 227243, 455, 220179, 480, 482, 484 ],
    ['Temperature', 224027, 645, 8537, 676, 677, 223762, 678, 679, 223761],
    ['Weight', 581],
    ['pH', 1126, 780, 223830, 220274, 220734, 4753, 4202, 1365, 7717, 3839]
]

labevents_codes = [
    ['Oxygen Saturation', 50817],
    ['Temperature', 50825],
    ['pH', 50820],
    ['% Hemoglobin A1c', 50852, 50854],
    ['Blood Glucose', 50931, 51529],
    ['Serum Creatinine', 50912]    
]

relevant_chartevents_codes = [
    ['Glucose', 3744, 3745, 1310, 807, 1529, 811, 220621, 226537, 3447, 225664],
    ['Heart Rate', 211, 220045],
    ['Mean blood pressure', 225312, 52, 6702, 220052, 6927, 3312, 3314, 3316, 7618, 3318, 3320, 3322, 7620, 7622, 3324, 5702, 443, 456, 220181],
    ['Oxygen Saturation', 0],
    ['Systolic blood pressure', 51, 225309, 220050, 3313, 3315, 3317, 3319, 3321, 3323, 3325, 442, 224167, 227243, 455, 220179, 480, 482, 484 ],    
    ['Temperature', 224027, 645, 8537, 676, 677, 223762, 678, 679, 223761],
    ['Weight', 581]
]

relevant_labevents_codes = [
    ['Oxygen Saturation', 50817],
    ['Temperature', 50825],
    ['Blood Glucose', 50931, 51529], 
]

In [8]:
def create_codes_table():
    cursor.execute("SHOW TABLES LIKE 'featurescodes'")
    if (cursor.fetchone() is None):
        try:
            query = "create external table default.featurescodes  (code int, mimiciiitable string, feature string, icd9code int) stored as PARQUET location 's3://aws-athena-query-results-067114122515-us-east-1/featurescodes'"
            cursor.execute(query)
            cnt = 0
            for f in labevents_codes:
                fn = f[0]
                for c in f[1:]:
                    query = "insert into featurescodes values (" + str(cnt) + ",'labevents','" + fn +"'," + str(c) +")"
                    cursor.execute(query)
                    cnt += 1
            for f in chartevents_codes:
                fn = f[0]
                for c in f[1:]:
                    query = "insert into featurescodes values (" + str(cnt) + ",'chartevents','" + fn +"'," + str(c) +")"
                    cursor.execute(query)
                    cnt += 1
            print ("featurescodes table created!")
        except Exception as e:
            print (e)
    else:
        print ("featurescodes table already exists.")
            

### Featurescodes table creation

In [9]:
#chartevents_codes = relevant_chartevents_codes
#labevents_codes = relevant_labevents_codes
create_codes_table()

featurescodes table created!


Item codes are unique among chart and lab events.

In [10]:
cursor.execute("select count(*) from featurescodes f, featurescodes g where g.icd9code=f.icd9code and g.feature <> f.feature")
assert 0 == cursor.fetchone()[0], "Different features have same code in featurescodes table!!!"

## Create joint events table

This table holds the chart events AND the lab events that correspond to the selected codes from the patients in the cohort.

We only keep here the events that were taken ** *48 hours before discharge* **.

In [11]:
def create_joint_events():
    cursor.execute("""CREATE TABLE events AS 
                    SELECT p.""" + subject_id_col + """ as subject_id, e.itemid,
                             e.charttime,
                             e.valuenum,
                             p.admit_time,
                             p.discharge_time,
                             p.mortality_flag
                    FROM """ + cohort_table + """ p
                    LEFT OUTER JOIN                              
                    (SELECT subject_id,
                             itemid,
                             charttime,
                             valuenum
                    FROM mimiciii.labevents l
                    INNER JOIN default.featurescodes f
                        ON f.icd9code = l.itemid                    
                    WHERE f.mimiciiitable='labevents'
                    UNION
                    SELECT subject_id,
                             itemid,
                             charttime,
                             valuenum
                    FROM mimiciii.chartevents c
                    INNER JOIN default.featurescodes f
                        ON f.icd9code = c.itemid                      
                    WHERE f.mimiciiitable='chartevents') e ON e.subject_id=p.subject_id
                    AND e.charttime < p.discharge_time - INTERVAL '48' HOUR
                    ORDER BY  p.""" + subject_id_col + """, e.itemid, e.charttime
    """)
    

In [12]:
cursor.execute("SHOW TABLES LIKE 'events'")
if (cursor.fetchone() is None):
    create_joint_events()

### Create events_daystodischarge table

This table adds the days to discharge date of the patients for each event.  

This is needed for further grouping of the events into individual dates.

In [13]:
def create_events_daystodischarge():
    cursor.execute("""CREATE TABLE events_daystodischarge AS SELECT subject_id,
                             -date_diff('day', discharge_time, charttime) daystodischarge, itemid, valuenum, mortality_flag
                    FROM events e
                    ORDER BY  subject_id, daystodischarge desc, itemid""")

In [14]:
cursor.execute("SHOW TABLES LIKE 'events_daystodischarge'")
if (cursor.fetchone() is None):
    create_events_daystodischarge()

### Create events_feature table

This table holds the average of the measurements taken for the same feature on the same day for each patient.

We took the average of the valuenum for each itemid on each day

In [15]:
def create_events_features():
    cursor.execute("""CREATE TABLE events_features 
                    AS SELECT subject_id,
                             daystodischarge,
                             f.code,
                             avg(valuenum) value,
                             mortality_flag
                    FROM events_daystodischarge e
                    LEFT JOIN featurescodes f ON e.itemid=f.icd9code
                    GROUP BY  subject_id, daystodischarge, f.code, mortality_flag
                    ORDER BY  subject_id, daystodischarge desc, code""")


In [16]:
cursor.execute("SHOW TABLES LIKE 'events_features'")
if (cursor.fetchone() is None):
    create_events_features()

## Sanity check

In [17]:
cursor.execute("""SELECT feature,
                         avg(value) avg,
                         stddev(value) dev,
                         min(value) min,
                         max(value) max,
                         count(value) cnt
                FROM events_features AS e
                INNER JOIN featurescodes AS f
                    ON e.code=f.code
                WHERE value>0 and value<1000
                GROUP BY  feature
                ORDER BY  dev DESC """)
df = as_pandas(cursor)
df

,feature,avg,dev,min,max,cnt
0,Blood Glucose,146.786787,66.943482,9.000000,981.000000,163775
1,Glucose,152.473772,55.603106,3.000000,952.500000,176479
2,Temperature,69.520109,30.686679,0.800000,300.020000,140187
3,Weight,90.057120,26.362641,0.300000,300.000000,25210
4,Systolic blood pressure,122.299468,20.378424,0.300000,964.375000,95847
5,Mean blood pressure,78.258917,14.859039,0.333333,891.857143,95618
6,Heart Rate,84.969130,14.838700,31.800000,230.566667,75725
7,Oxygen Saturation,90.092240,11.695140,1.400000,542.500000,20731


In [18]:
cursor.execute("""
                SELECT min(daystodischarge) minDays, max(daystodischarge) maxDays, avg(daystodischarge) avgDays, stddev(daystodischarge) stddevDays FROM events_features""")
df = as_pandas(cursor)
df

,minDays,maxDays,avgDays,stddevDays
0,2,4039,255.256692,557.691189


In [19]:
cursor.execute("""
                SELECT subject_id, count(distinct daystodischarge) cntDays FROM events_features GROUP BY subject_id""")
df = as_pandas(cursor)

In [21]:
cursor.execute("""
                SELECT mortality_flag, count(distinct """ +subject_id_col + """) cnt FROM """ + cohort_table + """ group by mortality_flag""")
df = as_pandas(cursor)
df

,mortality_flag,cnt
0,0,8605
1,1,1217


In [22]:
cursor.execute("""
                SELECT mortality_flag, count(*) cnt FROM events_features group by mortality_flag""")
df = as_pandas(cursor)
df

,mortality_flag,cnt
0,1,175660
1,0,661539


## Construction of Events dataframe

The data from the events_feature table are loaded into two dataframes.

The `events_item` dataframe contains for each patient and for each value of days-to-discharge a list of the features codes that were taken on that day.

The `events_values` dataframe contains form each patient and for each value of days-to-discharge a list of the values corresponding to the features in the `events_item`.

In [24]:
cursor.execute("select * from events_features order by subject_id, daystodischarge desc, code")
print ("Query finished")
events = as_pandas(cursor)

Query finished


In [25]:
num_pat = len(events['subject_id'].unique())

Non numbers are replaced with 0.

In [26]:
events['code'] = events['code'].fillna(0).astype(int)
events['daystodischarge'] = events['daystodischarge'].fillna(0).astype(int)
events['value'] = events['value'].fillna(0)

In [27]:
events_item = events.groupby([events['subject_id'], events['daystodischarge']])['code'].apply(list).reset_index(name='codes')
events_values = events.groupby([events['subject_id'], events['daystodischarge']])['value'].apply(list).reset_index(name='values')
assert len(events_item['subject_id'].unique()) == num_pat, 'Wrong number of patients in events_item'
assert len(events_values['subject_id'].unique()) == num_pat, 'Wrong number of patients in events_values'

Dataframes are *pickled*.

In [28]:
pickle.dump( events_item, open(prefix + "events_item.p", "wb" ) )
pickle.dump( events_values, open(prefix + "events_value.p", "wb"))
pickle.dump( patients, open(prefix + "patients.p", "wb"))

Just some assertions to make sure we saved what we expected.

In [ ]:
ei = pickle.load( open( prefix+"events_item.p", "rb" ) )

assert len(ei[ei['subject_id']==13]['codes']) == len(events_item[events_item['subject_id']==13]['codes']), "Wrong serialization!!"
assert len(ei)==len(events_item, ), "Wrong serialization!!"

ei = pickle.load(open(prefix+"events_value.p", "rb") )
#assert ei[ei['subject_id']==13]['values'][0] == events_values[events_values['subject_id']==13]['values'][0], "Wrong serialization!!"
assert len(ei)==len(events_item), "Wrong serialization!!"

ei = pickle.load(open(prefix+'patients.p', 'rb'))
assert int(ei[ei['subject_id']==2511]['mortality_flag']) == int(patients[patients['subject_id']==2511]['mortality_flag'])

## Auxiliary data

In [23]:
cursor.execute("select f.code, mimiciiitable, feature, avg(value) mean, stddev(value) stddev, count(value) cnt from featurescodes f left outer join events_features e on e.code=f.code  group by f.code, mimiciiitable, feature order by f.code")
print ("Features list")
features = as_pandas(cursor)
features.to_csv('featurescodes.csv')

Features list


In [29]:
cursor.execute('select max(code) from featurescodes')
max_code = int(cursor.fetchone()[0])

In [30]:
pickle.dump(max_code, open("events_maxcode.p", "wb"))
max_code

63

## Patients without data 

Probably because they were discharge in less than 48 hours.

In [32]:
dtod[dtod['daystodischarge']==0].sort_values(['subject_id'])

,daystodischarge
subject_id,
5532,0
5844,0
7132,0
8267,0
10879,0
11627,0
15583,0
22231,0
22840,0


In [33]:
print ("## Number of patients without data: ", len(dtod[dtod['daystodischarge']==0]))

## Number of patients without data:  23
